In [46]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

1- Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota, fecha). Se pide resolver utilizando PySpark:

A) Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
B) Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).
C) El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).


In [47]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [48]:
rdd = sc.textFile('spark-1.csv')
rdd.take(5)

['57,Algebra,8,20-08-17',
 '42,Algebra,8,16-06-17',
 '36,Fisica,6,18-06-19',
 '19,Fisica,3,30-03-18',
 '17,Algebra,8,29-01-17']

In [49]:
rdd.getNumPartitions()

2

A) Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.

In [50]:
rdd = rdd.map(lambda line: (int(line.split(',')[0]), line.split(',')[1], int(line.split(',')[2]), line.split(',')[3]))
rdd.take(5)

[(57, 'Algebra', 8, '20-08-17'),
 (42, 'Algebra', 8, '16-06-17'),
 (36, 'Fisica', 6, '18-06-19'),
 (19, 'Fisica', 3, '30-03-18'),
 (17, 'Algebra', 8, '29-01-17')]

In [51]:
rdd_reducido = rdd.filter(lambda x: x[2]>=4 and int(x[3].split('-')[2]) >= 18)
rdd_reducido.take(5)

[(36, 'Fisica', 6, '18-06-19'),
 (59, 'Fisica', 10, '23-04-20'),
 (22, 'Quimica', 4, '09-03-20'),
 (23, 'Quimica', 8, '19-09-18'),
 (57, 'Analisis', 10, '28-08-19')]

In [73]:
rdd_alumnos = rdd_reducido.map(lambda x: (x[0], 1)).distinct().count()
rdd_alumnos

23

B) Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón, promedio).

In [83]:
aTuple = (0,0)
probando = rdd.map(lambda x: (x[0],x[2])).aggregateByKey(aTuple, lambda a,b: (a[0] + b, a[1] + 1), 
                              lambda a,b: (a[0] + b[0], a[1] + b[1]))
final = probando.mapValues(lambda v: v[0]/v[1]).collect()
print(final)

[(42, 4.666666666666667), (36, 4.5), (24, 3.0), (20, 10.0), (12, 6.5), (10, 8.5), (14, 6.5), (22, 6.5), (54, 5.0), (58, 5.5), (2, 7.0), (48, 6.333333333333333), (50, 9.0), (32, 7.5), (52, 7.666666666666667), (46, 6.0), (38, 8.0), (18, 5.5), (8, 4.5), (56, 4.0), (34, 6.0), (6, 7.0), (64, 6.5), (57, 8.666666666666666), (19, 3.0), (17, 7.25), (33, 1.0), (37, 5.0), (61, 1.0), (59, 5.5), (23, 9.0), (39, 7.5), (7, 6.0), (35, 5.75), (1, 3.6666666666666665), (11, 2.0), (21, 4.6), (31, 5.0), (25, 10.0), (47, 10.0), (41, 1.0), (29, 8.5), (51, 8.5), (3, 7.0), (55, 1.0), (43, 7.0), (45, 5.0), (63, 2.0), (5, 4.0), (49, 5.0)]


First lambda expression for Within-Partition Reduction Step::
      a: is a TUPLE that holds: (runningSum, runningCount).
      b: is a SCALAR that holds the next Value

      Second lambda expression for Cross-Partition Reduction Step::
      a: is a TUPLE that holds: (runningSum, runningCount).
      b: is a TUPLE that holds: (nextPartitionsSum, nextPartitionsCount).

C) El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un segundo RDD alumnos con registros (padron, nombre y apellido).

In [87]:
rdd_nombres = sc.textFile('dataOct-21-2020.csv')
rdd_nombres.take(5)

['10,Dorian,Erickson',
 '1,Stephen,Mcbride',
 '4,Denton,Head',
 '6,Cameran,Bond',
 '3,Kim,Reilly']

In [95]:
final_rdd = sc.parallelize(final)
final_rdd.reduce(lambda a,b: a if a[1]>b[1] else b)

(47, 10.0)